In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Vertex%20AI%20Pipeline&download_url=https%3A%2F%2Fraw.githubusercontent.com%2Fkweinmeister%2Fnotebooks%2Fmaster%2Fcausal_inference_with_vertex_ai_automl_forecasting.ipynb">
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="Google Cloud Notebooks"> Open in GCP Notebooks
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/kweinmeister/notebooks/blob/master/causal_inference_with_vertex_ai_automl_forecasting.ipynb.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/kweinmeister/notebooks/blob/master/causal_inference_with_vertex_ai_automl_forecasting.ipynb.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# Causal Inference with Vertex AI AutoML Forecasting

The goal of this notebook is to demonstrate how to estimate the effect of a treatment. A treatment, or intervention, is an action intended to change an outcome. For example, you could estimate the effect of a marketing campaign or promotion on sales.

We'll look at answering the question, "How did Brexit impact exchange rates between the British Pound and US Dollar?" We'll use time-series data from [FRED](https://fred.stlouisfed.org).

This notebook will use the tabular data forecasting service from [Vertex AI](https://cloud.google.com/vertex-ai):
* We will train a time-series forecasting model on the British Pound and a related time-series (Euro) to generate a counterfactual.
* The counterfactual time series aims to answer the question "What would the value of the Pound be, had this event not happened?"
* We will then predict the exchange rate during the 30 day period after the Brexit vote on June 23, 2016.
* The difference between the actual and counterfactual time series gives us an estimate of the vote.

Finally, we'll use [tfcausalimpact](https://github.com/WillianFuks/tfcausalimpact) to compare our results to a statistical approach. tfcausalimpact is a Python port of the R-based [CausalImpact](https://google.github.io/CausalImpact/), using [TensorFlow Probability](https://www.tensorflow.org/probability).

Citation:
* Board of Governors of the Federal Reserve System (US), retrieved from FRED, Federal Reserve Bank of St. Louis:
  * [UK / US Exchange Rate - DEXUSUK](https://fred.stlouisfed.org/series/DEXUSUK)
  * [Euro / US Exchange Rate - DEXUSEU](https://fred.stlouisfed.org/series/DEXUSEU)

## Imports

In [ ]:
# Install google-cloud-aiplatform (and other packages to match)

!pip3 install -U --quiet google-cloud-aiplatform google-cloud-bigquery grpcio pandas-gbq gcsfs tfcausalimpact folium==0.2.1 imgaug==0.2.6 tensorflow==2.5

# Restart runtime

import IPython
app = IPython.Application.instance() 
app.kernel.do_shutdown(True)

In [ ]:
# Import packages

from causalimpact import CausalImpact
from datetime import timedelta
from scipy import stats

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

## Constants

In [ ]:
# Set GCP project constants

PROJECT_ID = 'YOUR-PROJECT-ID' # Your project ID
REGION = 'us-central1' # Your project region
STORAGE_PATH = 'gs://YOUR-REGIONAL-BUCKET/data/exchange_rates' # A bucket and path to store various CSV files used (training, forecast). Bucket must already exist.

In [ ]:
# Set forecasting constants (no need to change these)

# Training data prior to event
PRE_PERIOD=[pd.to_datetime('2012-01-03'), pd.to_datetime('2016-06-23')]

# Duration of period after the event to analyze
POST_PERIOD_LENGTH=28
POST_PERIOD=[pd.to_datetime('2016-06-24'), pd.to_datetime('2016-07-22')]

# Lookback window used for forecasting (usually 1-5x the forecast horizon)
CONTEXT_WINDOW = 1 * POST_PERIOD_LENGTH

In [ ]:
# Other constants (no need to change these)

SCENARIO = 'exchange-rates'
TRAIN_CSV = 'train.csv'
FORECAST_INPUT_CSV = 'forecast_input.csv'

## Setup Google Cloud project

In [ ]:
# Authenticate to Google Cloud

if 'google.colab' in sys.modules: 
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
# Initialize Vertex AI SDK

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

!gcloud config set project $PROJECT_ID

## Read data

We will import two related time-series from [FRED](https://fred.stlouisfed.org/) (Federal Reserve Economic Data). One is the history of the US Dollar : British Pound exchange rate, and the other is of the US Dollar : Euro.  

In [ ]:
def read_fred_data(dataset, start_date, end_date):
  url = f'https://fred.stlouisfed.org/graph/fredgraph.csv?id={dataset}&cosd={str(start_date)[:10]}&coed={str(end_date)[:10]}'
  return pd.read_csv(url, index_col='DATE', parse_dates=True, na_values='.')

df_DEXUSUK=read_fred_data('DEXUSUK', PRE_PERIOD[0], POST_PERIOD[1])
df_DEXUSEU=read_fred_data('DEXUSEU', PRE_PERIOD[0], POST_PERIOD[1])

# Merge each series into one dataframe
df = pd.merge(left=df_DEXUSEU, left_on=df_DEXUSEU.index, right=df_DEXUSUK, right_on=df_DEXUSUK.index).rename(columns={'key_0':'DATE'}).set_index('DATE').dropna()

df.head()

## Visualize data

Let's now plot both time series: British Pound (`DEXUSUK`) and Euro (`DEXUSEU`).

The shaded area is the four-week "post period," which indicates a drop after the Brexit vote on June 23, 2016.

We will later generate "what would have been" (counterfactual) by learning from the proxy Euro time series.


In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(20)
ax.plot(df[POST_PERIOD[1] - timedelta(days=730):POST_PERIOD[1]])
ax.axvspan(POST_PERIOD[0], POST_PERIOD[1], alpha=0.5, color='lightblue')

plt.title('GBP/US and Euro/US Exchange Rates')
plt.legend(df.columns)
plt.show()

Calculate the correlation between time series, to validate that an appropriate proxy is being used.

In [ ]:
corr, _ = stats.pearsonr(x=df.DEXUSEU[df.index <= PRE_PERIOD[1]], y=df.DEXUSUK[df.index <= PRE_PERIOD[1]])

print(f'Pearson correlation between time series in the pre-period: {round(corr, 2)}')

## Create and upload data files

We will now create and upload CSV files in the format that Vertex AI expects for training and forecasting.

In [ ]:
# Training data

df_train = df[df.index <= PRE_PERIOD[1]].copy()

# There is only one series, so we will set the series ID to 0 for all rows.
df_train['ID'] = 0

df_train.to_csv(TRAIN_CSV)

!gsutil cp {TRAIN_CSV} {STORAGE_PATH}/{TRAIN_CSV}

In [ ]:
# Forecast input: create dataframe

# Start with treatment dataframe
df_forecast_input = df.copy()

df_forecast_input['ID'] = 0

# Set sales to empty during treatment period (to be predicted)
df_forecast_input.loc[df_forecast_input.index >= POST_PERIOD[0], 'DEXUSUK'] = np.NaN

df_forecast_input

In [ ]:
# Forecast input: upload

df_forecast_input.to_csv(FORECAST_INPUT_CSV)

!gsutil cp {FORECAST_INPUT_CSV} {STORAGE_PATH}/{FORECAST_INPUT_CSV}

In [ ]:
# Create dataset


ds = dataset = aiplatform.TimeSeriesDataset.create(
    display_name=f'dataset-{SCENARIO}',
    gcs_source=f'{STORAGE_PATH}/{TRAIN_CSV}',
)

ds.resource_name

In [ ]:
# Train forecasting job

job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=f'model-{SCENARIO}',
    column_transformations=[
        {"numeric": {"column_name": "DEXUSUK"}},
        {"timestamp": {"column_name": "DATE"}},
    ],
    optimization_objective='minimize-rmse'
)

# This may take several hours to run
model = job.run(
    dataset=ds,
    target_column='DEXUSUK',
    time_column='DATE',
    time_series_identifier_column='ID',
    unavailable_at_forecast_columns=['DEXUSUK'],
    available_at_forecast_columns=['DEXUSEU', 'DATE'],
    data_granularity_unit='day',
    forecast_horizon=POST_PERIOD_LENGTH,
    data_granularity_count=1,
    context_window=CONTEXT_WINDOW,
    budget_milli_node_hours=8000
)

## Forecast with trained model

In [ ]:
# Try to access trained model from previous step.
# If one is not available (e.g. because the runtime has timed out), you can update YOUR-MODEL-ID below to use an already trained model.

try:
  model
except:
  model = aiplatform.Model('YOUR-MODEL-ID') # Example: 7354146250200646848

In [ ]:
# Perform a batch prediction

batch_prediction_job = model.batch_predict(
    job_display_name=f'prediction-{SCENARIO}',
    gcs_source=f'{STORAGE_PATH}/{FORECAST_INPUT_CSV}',
    gcs_destination_prefix=f'{STORAGE_PATH}',
    instances_format='csv',
    predictions_format='csv'
)

In [ ]:
# The job output provides the full path with the timestamp where the output CSV was generated.
# You can also look up this by looking at the batch prediction job details in the UI.

try:
  batch_prediction_job
except:
  batch_prediction_job = aiplatform.BatchPredictionJob('YOUR-BATCH-PREDICTION-JOB-ID')  # Example: 7296593272910163968

FORECAST_OUTPUT_FILE = f'{batch_prediction_job.output_info.gcs_output_directory}/predictions_1.csv'

df_forecast_output = pd.read_csv(FORECAST_OUTPUT_FILE, index_col='DATE', parse_dates=True).sort_values('DATE')

df_forecast_output.head()

In [ ]:
# We will now create a dataframe that contains the actual and predicted values for comparison.

df_estimate = df[['DEXUSUK']]

df_counterfactual = df_estimate.copy().rename(columns={'DEXUSUK':'predicted_DEXUSUK'})
df_counterfactual.loc[df_counterfactual.index.isin(df_estimate.index), 'predicted_DEXUSUK'] = df_forecast_output['predicted_DEXUSUK']

df_estimate = pd.merge(df_estimate, df_counterfactual, on='DATE')
df_estimate

## Plot the forecast

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(20)
ax.plot(df_estimate[df_estimate.index >= POST_PERIOD[0] - timedelta(days=3 * POST_PERIOD_LENGTH)])
ax.fill_between(df_estimate.index, df_estimate.DEXUSUK, df_estimate.predicted_DEXUSUK, color='lightblue')

plt.title('Comparison of actual and predicted US/UK exchange rate')
plt.legend(df_estimate.columns)
plt.show()

## Calculate summary statistics

In [ ]:
df_post_period = df_estimate[df_estimate.index >= POST_PERIOD[0]]

mean_actual = df_post_period.DEXUSUK.mean()
mean_predicted = df_post_period.predicted_DEXUSUK.mean()

print(f'Actual US/UK exchange rate:    {round(mean_actual, 3)}') 
print(f'Predicted US/UK exchange rate: {round(mean_predicted, 3)}')

auc = (df_post_period.DEXUSUK - df_post_period.predicted_DEXUSUK).sum()
ate = auc / len(df_post_period.index)

print(f'Average treatment effect:     {round(ate, 3)}')
print(f'Relative treatment effect:    {round(ate/mean_predicted * 100, 3)}%')

### Estimate effect with tfcausalimpact

We've already gone through data preprocessing steps, so it should be straightforward to pass in inputs into the [tfcausalimpact library](https://github.com/WillianFuks/tfcausalimpact), which uses [TensorFlow Probability](https://www.tensorflow.org/probability). It provides prediction intervals, to enable hypothesis testing on the significance of the treatment effect. 

In [ ]:
ci = CausalImpact(df[['DEXUSUK', 'DEXUSEU']], PRE_PERIOD, POST_PERIOD)
print(ci.summary())
print(ci.summary(output='report'))
ci.plot()

## Conclusion

In this notebook, we walked through how to estimate the effect of an intervention. We used 2 methods to generate the counterfactual, or synthetic control, that allows us to compare the outcomes with and without the treatment.